# EM HCP-Brick (Poisson) — End-to-End

Notebook generado automáticamente.

## 1) Generación de datos mock

In [ ]:
import numpy as np
import pandas as pd
import math

def simulate_hcp_brick_poisson(
    n_hcps=400,
    n_bricks=80,
    avg_bricks_per_hcp=2.5,
    seed=42,
):
    rng = np.random.default_rng(seed)

    specialties = np.array(["GP", "CARDIO", "ENDO", "ONCO", "PULMO"])
    spec = rng.choice(specialties, size=n_hcps, p=[0.45, 0.18, 0.15, 0.12, 0.10])

    seniority = rng.integers(0, 21, size=n_hcps)  # 0..20
    potential = rng.lognormal(mean=0.0, sigma=0.6, size=n_hcps)

    spec_df = pd.get_dummies(spec, prefix="spec")

    X = np.column_stack([
        np.ones(n_hcps),                # intercept
        seniority / 10.0,               # scaled
        np.log1p(potential),            # log feature
        spec_df.to_numpy(),             # dummies (note: collinearity with intercept -> use ridge)
    ])
    feature_names = (["intercept", "seniority_x0.1", "log1p_potential"]
                     + list(spec_df.columns))

    p = X.shape[1]

    beta_true = np.zeros(p)
    beta_true[0] = -0.3
    beta_true[1] =  0.20
    beta_true[2] =  0.55
    for j, nm in enumerate(feature_names):
        if nm == "spec_CARDIO":
            beta_true[j] = 0.25
        elif nm == "spec_ONCO":
            beta_true[j] = 0.35
        elif nm == "spec_PULMO":
            beta_true[j] = 0.10
        elif nm == "spec_ENDO":
            beta_true[j] = 0.15

    lambda_true = np.exp(X @ beta_true)

    # Membership
    K = rng.poisson(lam=avg_bricks_per_hcp, size=n_hcps)
    K = np.clip(K, 1, max(1, n_bricks // 2))

    brick_pop = rng.lognormal(mean=0.0, sigma=0.7, size=n_bricks)
    brick_pop = brick_pop / brick_pop.sum()

    rows = []
    for h in range(n_hcps):
        bricks_h = rng.choice(n_bricks, size=K[h], replace=False, p=brick_pop)
        base = rng.poisson(lam=2.5)
        for b in bricks_h:
            visits = base + rng.poisson(lam=0.15 * seniority[h] + 1.0)
            w = float(visits) if visits > 0 else 0.5
            rows.append((h, b, visits, w))

    hb = pd.DataFrame(rows, columns=["hcp_id", "brick_id", "visits", "w"])

    mu = lambda_true[hb["hcp_id"].to_numpy()] * hb["w"].to_numpy()
    hb["z_true"] = rng.poisson(lam=mu)

    y_by_brick = hb.groupby("brick_id")["z_true"].sum().rename("y").reset_index()

    hcp = pd.DataFrame({
        "hcp_id": np.arange(n_hcps),
        "specialty": spec,
        "seniority": seniority,
        "potential": potential,
        "lambda_true": lambda_true,
    })
    for j, nm in enumerate(feature_names):
        hcp[nm] = X[:, j]

    meta = {"feature_names": feature_names, "beta_true": beta_true}
    return hcp, hb, y_by_brick, meta

hcp_df, hb_df, y_df, meta = simulate_hcp_brick_poisson(seed=42)
hcp_df.head(), hb_df.head(), y_df.head(), meta["feature_names"], meta["beta_true"][:6]


## 2) EM desde cero (vectorizado + IRLS en M-step)

In [ ]:
import numpy as np
import math

def prepare_arrays(hcp_df, hb_df, y_df, feature_names):
    brick_ids = np.sort(y_df["brick_id"].unique())
    brick_map = {bid: i for i, bid in enumerate(brick_ids)}
    B = len(brick_ids)

    h = hb_df["hcp_id"].to_numpy().astype(int)
    b = hb_df["brick_id"].map(brick_map).to_numpy().astype(int)
    w = hb_df["w"].to_numpy().astype(float)

    y = np.zeros(B, dtype=float)
    idx = y_df["brick_id"].map(brick_map).to_numpy().astype(int)
    y[idx] = y_df["y"].to_numpy().astype(float)

    X = hcp_df[feature_names].to_numpy().astype(float)
    return X, h, b, w, y, brick_ids

def poisson_loglik_brick(y, mu):
    lgamma = np.vectorize(math.lgamma)
    return float(np.sum(y * np.log(mu + 1e-12) - mu - lgamma(y + 1.0)))

def irls_poisson_ridge(X, y, offset=None, alpha=1e-3, max_iter=50, tol=1e-8):
    n, p = X.shape
    if offset is None:
        offset = np.zeros(n, dtype=float)
    beta = np.zeros(p, dtype=float)

    for _ in range(max_iter):
        eta = X @ beta + offset
        mu = np.exp(eta)

        W = mu
        z = eta + (y - mu) / (mu + 1e-12)

        sqrtW = np.sqrt(W)
        Xw = X * sqrtW[:, None]
        yw = (z - offset) * sqrtW

        A = Xw.T @ Xw + alpha * np.eye(p)
        rhs = Xw.T @ yw

        beta_new = np.linalg.solve(A, rhs)
        if np.linalg.norm(beta_new - beta) / (np.linalg.norm(beta) + 1e-12) < tol:
            beta = beta_new
            break
        beta = beta_new

    return beta

def em_poisson_hcp_brick(
    X, h, b, w, y,
    alpha=1e-2,
    max_em_iter=200,
    tol=1e-6,
    mstep_max_iter=50,
    verbose=True,
):
    n_hcps, p = X.shape
    B = int(np.max(b)) + 1

    w_tilde = np.bincount(h, weights=w, minlength=n_hcps).astype(float)
    offset = np.log(w_tilde + 1e-12)

    beta = np.zeros(p, dtype=float)
    history = {"loglik": [], "rel_beta_change": []}

    for t in range(max_em_iter):
        # E-step
        lambda_h = np.exp(X @ beta)
        num = lambda_h[h] * w
        den = np.bincount(b, weights=num, minlength=B).astype(float)

        zhat = y[b] * (num / (den[b] + 1e-12))

        # M-step
        y_tilde = np.bincount(h, weights=zhat, minlength=n_hcps).astype(float)
        beta_new = irls_poisson_ridge(X, y_tilde, offset=offset, alpha=alpha, max_iter=mstep_max_iter)

        ll = poisson_loglik_brick(y, den)
        rel = float(np.linalg.norm(beta_new - beta) / (np.linalg.norm(beta) + 1e-12))
        history["loglik"].append(ll)
        history["rel_beta_change"].append(rel)

        if verbose and (t % 10 == 0 or t < 5):
            print(f"EM iter {t:3d} | loglik={ll: .3f} | rel_beta_change={rel: .3e}")

        if rel < tol:
            beta = beta_new
            break
        beta = beta_new

    # final E-step
    lambda_h = np.exp(X @ beta)
    num = lambda_h[h] * w
    den = np.bincount(b, weights=num, minlength=B).astype(float)
    zhat = y[b] * (num / (den[b] + 1e-12))
    return beta, history, zhat

X, h, b, w, y, brick_ids = prepare_arrays(hcp_df, hb_df, y_df, meta["feature_names"])
beta_hat, hist, zhat = em_poisson_hcp_brick(X, h, b, w, y, alpha=1e-2, verbose=True)
beta_hat[:8], meta["beta_true"][:8], len(hist["loglik"]), hist["loglik"][-1]


## 3) Evaluación contra ground truth

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

beta_true = meta["beta_true"]

param_df = pd.DataFrame({
    "feature": meta["feature_names"],
    "beta_true": beta_true,
    "beta_hat": beta_hat,
    "diff": beta_hat - beta_true,
})
display(param_df.head(12))

eta_true = X @ beta_true
eta_hat = X @ beta_hat
corr_eta = np.corrcoef(eta_true, eta_hat)[0, 1]
rmse_eta = float(np.sqrt(np.mean((eta_true - eta_hat) ** 2)))
print("corr(eta_true, eta_hat) =", corr_eta)
print("rmse(eta_true, eta_hat) =", rmse_eta)

hb_eval = hb_df.copy()
hb_eval["zhat"] = zhat
corr_z = np.corrcoef(hb_eval["z_true"], hb_eval["zhat"])[0, 1]
rmse_z = float(np.sqrt(np.mean((hb_eval["z_true"] - hb_eval["zhat"]) ** 2)))
print("corr(z_true, zhat) =", corr_z)
print("rmse(z_true, zhat) =", rmse_z)

brick_sum = hb_eval.groupby("brick_id")[["z_true", "zhat"]].sum().reset_index()
brick_sum = brick_sum.merge(y_df, on="brick_id", how="left")
brick_sum["zhat_minus_y"] = brick_sum["zhat"] - brick_sum["y"]
print("max |sum_h zhat - y| =", float(brick_sum["zhat_minus_y"].abs().max()))

plt.figure()
plt.plot(hist["loglik"])
plt.xlabel("EM iteration")
plt.ylabel("Observed log-likelihood")
plt.title("EM convergence (log-likelihood)")
plt.show()


## Debugging típico de EM (checklist)

1) **Suma por brick**: `sum_h zhat_{h,b}` debe ser exactamente `y_b` (errores ~1e-9).  
2) **Denominadores**: `den[b]` no debe ser 0; si lo es, bricks sin HCPs o `w=0`.  
3) **Colinealidad**: intercept + todas las dummies => usa ridge (`alpha>0`) o elimina una dummy.  
4) **Offsets**: `w_tilde` debe ser >0 para todos los HCPs incluidos (si no, filtra).  
5) **Overflow**: `exp(X beta)` puede desbordar => estandariza/clip `eta` o sube regularización.  
6) **Monotonicidad**: loglik observada no debería bajar; si baja, suele ser bug en el M-step.  
7) **Convergencia falsa**: si beta no cambia pero loglik es mala, revisa inicialización/alpha.
